In [ ]:
import consts, test_helper
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn, scipy, requests

In [ ]:
importlib.reload(consts)
importlib.reload(test_helper)

In [ ]:
# USER
USER = "RYAN"
FILE_PATH = consts.PATH_MAP[USER]

ROW = consts.ROW
COL = consts.COL
CLEANED_DATA_PATH = consts.CLEANED_DATA_PATH
DATA_PATH = consts.RAW_DATA_PATH
RESPONSE_NAME = consts.RESPONSE_NAME

TRAIN_START_DATE = "20150101"
TRAIN_END_DATE = "20150601" # Up to but not including
TEST_START_DATE = "20150701"
TEST_END_DATE = "20150801"

LASSO = 'LASSO'
XGBOOST = 'xgboost'

# REGRESSION_TYPES = helper.Regression('OLS').list_all_regression_types()
# REGRESSION_TYPES

In [ ]:
list_of_interacting_terms = [["relvol_nt_0","rrirpnxm_nt_0"], 
                            ["relvol_lst15_0", "rrirpnxm_lst15_0"],
                            ["relvol_lsthrx15_0", "rrirpnxm_lsthrx15_0"],
                            ["relvol_toxhr_0", "rrirpnxm_toxhr_0"]]

FEATURE_COL_NAMES = ["rrirpnxm_nt_0", "rrirpnxm_lst15_0", "rrirpnxm_lsthrx15_0", "rrirpnxm_toxhr_0"]

In [ ]:
importlib.reload(test_helper)
data = test_helper.Data(FILE_PATH)
train_df = data.update_and_get_train_df(FILE_PATH, "20190701", movingBack_dayCount = 31, years_count = 1)

In [ ]:
importlib.reload(test_helper)
test_dfs = data.get_df_between_date(data_path = FILE_PATH, start_date="20150701", end_date="20150801")

In [ ]:
importlib.reload(test_helper)
ols_model = test_helper.Regression(data_path = FILE_PATH)
ols_model.train(train_df, feature_col_names = FEATURE_COL_NAMES, interacting_terms_list = list_of_interacting_terms, hyperparam_dict={'fit_intercept': False, 'positive':True})
ols_model.get_metric(dataframes = test_dfs)

In [ ]:
importlib.reload(test_helper)
full_ols_model = test_helper.Regression(data_path = FILE_PATH)
full_ols_model.train(train_df)
full_ols_model.get_metric(test_dfs)

In [ ]:
importlib.reload(test_helper)
hyperparams = {'max_iter':900}
training_params = {'sample_weight': None}
lasso_model = test_helper.Regression(regression_type="Lasso", data_path=FILE_PATH, hyperparam_dict=hyperparams )
lasso_model.train(train_df, feature_col_names = FEATURE_COL_NAMES, interacting_terms_list = list_of_interacting_terms)
lasso_model.get_metric(dataframes = test_dfs)

In [ ]:
importlib.reload(test_helper)
hyperparams = {'max_iter':900}
training_params = {'sample_weight': None}
test_helper.experiment(LASSO, FILE_PATH, train_df=train_df, test_dfs=test_dfs, features=FEATURE_COL_NAMES, model_hyperparams=hyperparams, training_hyperparams=training_params, interacting_terms=list_of_interacting_terms)
#(0.02999211607260984, 0.00022296778636828244, 0.8806654508326743)

In [ ]:
xgb_model = test_helper.Regression(regression_type="XGBoost", data_path=FILE_PATH, hyperparam_dict={'max_depth': 6})
xgb_model.train(train_df, feature_col_names = FEATURE_COL_NAMES, interacting_terms_list = list_of_interacting_terms)
xgb_model.get_metric(test_dfs)
# Equivalent to
test_helper.experiment(XGBOOST, FILE_PATH, 
                    train_df=train_df,
                    test_dfs=test_dfs,
                    features=FEATURE_COL_NAMES,
                    interacting_terms=list_of_interacting_terms,
                    model_hyperparams={'max_depth':6},
                    training_hyperparams=None)


In [ ]:
# Itereate over hyperparameter :
importlib.reload(test_helper)
metrics = test_helper.iterate_hyperparam(XGBOOST, hyperparam_name='max_depth', hyperparam_values=[int(x) for x in np.linspace(1,10,10)], data_path=FILE_PATH, train_df=train_df, test_dfs=test_dfs, features=FEATURE_COL_NAMES,interacting_terms=list_of_interacting_terms )
mean_returns = metrics[1]
plt.scatter(list(mean_returns.keys()),list(mean_returns.values()))
plt.show()
